In [1]:
import networkx as nx
import numpy as np

def get_ba_with_degree_cap(N, m=2, max_degree=10, seed=0):
    """
    Modified Barabási-Albert model with maximum degree constraint.
    """
    np.random.seed(seed)
    G = nx.empty_graph(m)  # Start with m isolated nodes
    targets = list(range(m))
    repeated_nodes = list(targets)

    for new_node in range(m, N):
        # Filter eligible targets (nodes with degree < max_degree)
        eligible = [node for node in repeated_nodes if G.degree[node] < max_degree]
        eligible = list(set(eligible))  # Remove duplicates

        # If not enough eligible nodes, connect to random nodes
        if len(eligible) < m:
            targets = np.random.choice(list(G.nodes), size=m, replace=False)
        else:
            probs = np.array([G.degree[node] for node in eligible], dtype=float)
            probs = probs/probs.sum() if probs.sum() > 0 else np.ones(len(eligible))/len(eligible)
            targets = np.random.choice(eligible, size=m, replace=False, p=probs)

        for target in targets:
            G.add_edge(new_node, target)

        # Update repeated_nodes list
        repeated_nodes.extend(targets)
        repeated_nodes.extend([new_node] * m)

    return nx.to_numpy_array(G, dtype=int)

adj = get_ba_with_degree_cap(800, m=1, max_degree=2, seed=0)
adj

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [2]:
neighbor_list = []
for i in range(adj.shape[0]):
    neighbors = np.where(adj[i] == 1)[0]
    neighbor_list.append(neighbors.tolist())
print("Number of nodes:", len(neighbor_list))
print("Number of edges:", sum(len(neighbors) for neighbors in neighbor_list) // 2)
print("Average degree:", sum(len(neighbors) for neighbors in neighbor_list) / len(neighbor_list))
print("Max degree:", max(len(neighbors) for neighbors in neighbor_list))
print("Min degree:", min(len(neighbors) for neighbors in neighbor_list))

Number of nodes: 800
Number of edges: 799
Average degree: 1.9975
Max degree: 2
Min degree: 1


In [33]:
from dgp_old import *

# 1. Simulate or load data
sample_size = 800
m = 2
max_degree = 4
adj = adj = get_ba_with_degree_cap(sample_size, m=m, max_degree=max_degree, seed=0)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=30,
    burnin_R=10, seed=1)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 40/40 [00:01<00:00, 30.30it/s]


In [34]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=10,
    seed=0
)

{'average': 0.260475,
 'direct_effect': -0.14177499999999996,
 'spillover_effect': -0.16287500000000016,
 'psi_1_gamma': 0.216075,
 'psi_0_gamma': 0.35784999999999995,
 'psi_zero': 0.5207250000000001}

In [35]:
from drnet import doubly_robust
from run_pll import cols_raw

ret_i = doubly_robust(A_chain[-2], L_chain[-2], Y_chain[-2], adj, return_raw=False)
ret_i

[[0 1 1 ... 1 1 1]
 [0 1 0 ... 1 1 0]
 [1 1 1 ... 1 0 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 0]
 [1 0 1 ... 1 0 1]]
numerator_vec: [[0.32155661 0.69920761 1.42978176 ... 0.6878902  1.77069895 1.42978176]
 [0.42327377 0.51331777 0.37008277 ... 0.79376064 0.3979092  0.37008277]
 [1.14102088 1.26158435 1.14225205 ... 2.44992418 0.7813701  1.14225205]
 ...
 [1.00898246 1.93865039 1.93865039 ... 1.00898246 1.00898246 1.93865039]
 [1.28043312 1.62069028 2.46021339 ... 1.28043312 0.84349818 0.95556888]
 [3.38599474 2.75587556 3.38599474 ... 3.38599474 2.75587556 1.03378755]]


{'average': 0.27934586185323396,
 'direct_effect': -0.12690004843536176,
 'spillover_effect': -0.2160878206657013,
 'psi_1_gamma': 0.2419761704442089,
 'psi_0_gamma': 0.36887621887957067,
 'psi_zero': 0.584964039545272}

In [36]:
from drnet_em import doubly_robust_em

ret_i_em = doubly_robust_em(A_chain[-3], L_chain[-3], Y_chain[-3], adj, return_raw=False)
ret_i_em

{'average': 0.24973979903611543,
 'direct_effect': -0.06422253186035845,
 'spillover_effect': -0.3629879873083496,
 'psi_1_gamma': 0.23166111366673256,
 'psi_0_gamma': 0.295883645527091,
 'psi_zero': 0.6588716328354406}

In [ ]:
num_sample = 30
num_burn = 10
sample_size = 800

from dgp_old import *

# 1. Simulate or load data
min_degree = 5
max_degree = 7
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 40/40 [00:01<00:00, 27.37it/s]


In [2]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=10,
    seed=0
)

{'average': 0.201,
 'direct_effect': 0.025250000000000022,
 'spillover_effect': -0.23697500000000005,
 'psi_1_gamma': 0.21157499999999999,
 'psi_0_gamma': 0.18632499999999996,
 'psi_zero': 0.4233}

In [13]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(0.009349342448493666, 0.4965)

In [2]:
n_cpu = 10

In [5]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x10808c1f0> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [6]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.18595 ± 0.03886
direct_effect: -0.00959 ± 0.04825
spillover_effect: -0.24056 ± 0.08832
psi_0_gamma: 0.19490 ± 0.05247
psi_zero: 0.43545 ± 0.06247
psi_1_gamma: 0.18530 ± 0.03915


In [7]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x164620a60> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [8]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.17153 ± 0.03228
direct_effect: -0.02476 ± 0.02289
spillover_effect: -0.20472 ± 0.06280
psi_0_gamma: 0.19140 ± 0.03556
psi_zero: 0.39611 ± 0.03821
psi_1_gamma: 0.16664 ± 0.03403


In [3]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr_em = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr_em = np.array(res_list_dr_em)

Multiprocessing <function run_dr_em at 0x15c9e0ee0> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [4]:
from run_pll import column_names

ret_mean_dr_em = res_list_array_dr_em.mean(axis=0)
ret_std_dr_em = res_list_array_dr_em.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr_em[i]:.5f} ± {ret_std_dr_em[i]:.5f}")

average: 0.10687 ± 0.01503
direct_effect: 0.03032 ± 0.02366
spillover_effect: -0.19783 ± 0.07553
psi_0_gamma: 0.08564 ± 0.01672
psi_zero: 0.28347 ± 0.06859
psi_1_gamma: 0.11596 ± 0.01925
